In [1]:
%matplotlib inline

In [7]:
import numpy as np
from scipy.stats import scoreatpercentile
from nilearn.image import iter_img
from nilearn.plotting import plot_stat_map, show
from pylab import figure
import os

mask_img ='/storage/ADHD200/athena/NYU/ADHD200_mask_152_4mm.nii.gz'

from nilearn.input_data import NiftiMasker
masker = NiftiMasker(mask_img=mask_img, 
                     standardize=True,
                     detrend=1,
                     smoothing_fwhm=6.,
                     memory="/storage/nilearn_cache", 
                     memory_level=2)
masker.fit()

def flip(row):
    if np.sum(row > 0) < np.sum(row < 0):
        row *= -1
    return row
        
import statistics
# components_img1=stats.zscore(components_img1,axis=0)
# components_img2=stats.zscore(components_img2,axis=0)
# components_img3=stats.zscore(components_img3,axis=0)
def thresholding(array):    
    thr=[]
    array1=array
    
    for idx,row in enumerate(array):
        row=flip(row)  
        row[row < 0] = 0
        T=np.amax(row)*0.3
        row[np.abs(row) < T] = 0
        
#         T=scoreatpercentile(np.abs(row),P)
#         T=2.3                
#         T=max(row)*0.2
        
        row=row/np.std(row)        
        array1[idx,:]=row
    return array1

def save_maps(components_img,dir):        
    if not os.path.exists(dir):
        os.makedirs(dir)
    components_img = masker.inverse_transform(components_img)
    for i, cur_img in enumerate(iter_img(components_img)):
        outname=dir+str(i+1)+'.png'
        plot_stat_map(cur_img, display_mode="z", black_bg=True,cut_coords=1,annotate=False,
                     colorbar=False,output_file=outname)
        
#         if i==5:
#             break




/usr/local/lib/python3.5/dist-packages/nilearn/_utils/cache_mixin.py:84: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/storage/nilearn_cache'", use "location='/storage/nilearn_cache'" instead.
  memory = Memory(cachedir=cache_dir, verbose=verbose)


In [10]:
def vis(dirIn,dirOut):
    components_img1=np.load(dirIn)
    print(components_img1.shape)
    
    components_img1=thresholding(components_img1)    


    save_maps(components_img1,dir=dirOut)        
    
#     np.save('/home/share/TmpData/Qinglin/HCP_Group_DBN/'+task+'_map1.npy',components_img1)
#     np.save('/home/share/TmpData/Qinglin/HCP_Group_DBN/'+task+'_map2.npy',components_img2)
#     np.save('/home/share/TmpData/Qinglin/HCP_Group_DBN/'+task+'_map3.npy',components_img3)
    


In [12]:
vis('/home/uga_qinglin/Documents/ADHD_DL/ADHD200_VAE_components.npy','/home/uga_qinglin/Documents/ADHD_DL/VAEmaps/')


(40, 28546)


/usr/local/lib/python3.5/dist-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "
/usr/local/lib/python3.5/dist-packages/scipy/ndimage/measurements.py:272: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return _nd_image.find_objects(input, max_label)


In [13]:
vis('/home/uga_qinglin/Documents/ADHD_DL/ADHD200_DL_components.npy','/home/uga_qinglin/Documents/ADHD_DL/DLmaps/')

(40, 28546)
